# Instructions

This notebook takes an excel file of authors and their institutions and searches PubMed for tuberculosis-related papers from those authors published during a user-specified date range. Output is a semicolon-separated list of PubMed IDs which can be pasted directly into the PubMed search box to get a page listing those papers.

1. **Upload "20231114_SEATRAC_Member_PubSearch.xlsx"**:
    * Click the folder symbol on the left, then the upload symbol. If you get a warning about saving elsewhere click "ok".
    * If you're using a newer version ensure these columns still exist: 'Last Name', 'First Name', 'Middle Initial', 'Primary Institution (Choose ONE)'
2. **Update the search dates**:
    * In the "Inputs" code box below, edit the text within quotes
    * Click the play button to run this chunk of code, saving the inputs
3. **Do the search**:
    * Click the play button of the 'Do PubMed Search' code box (should take 1-2min to finish, output is at the bottom)

#### Extra info:

* PubMed API (NCBI Entrez E-utilities) help: https://www.ncbi.nlm.nih.gov/books/NBK25497/#chapter2.Introduction
* More help: https://pubmed.ncbi.nlm.nih.gov/help/
* List of PubMed "tags": https://www.ncbi.nlm.nih.gov/pmc/about/userguide/

# Inputs

Click the play button on the left after updating these. Be sure to use full double-digit numbers for the day and month.

In [47]:
start_date = '2024/04/01'
end_date = '2024/06/30'

excel_file = '/content/20231114_SEATRAC_Member_PubSearch.xlsx'

# Do PubMed Search

In [51]:
#########
# Setup #
#########

print('Loading packages and data')

from bs4 import BeautifulSoup
import requests
import pandas as pd

# Create URL retrieving a maximum of 10,000 records (retmax)
# You can include your ncbi API key '&api_key=XXXX'
prefix = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmax=10000&usehistory=y&term='

# Load input excel sheet
seatrac_df = pd.read_excel(excel_file)

# Clean up leading and trailing whitespace from columns where item is a string (not NaN)
seatrac_df = seatrac_df.map(lambda x: x.strip() if isinstance(x, str) else x)

####################################
# Format affiliations and concepts #
####################################

print('Formatting affiliations and concepts')

affiliations_list = seatrac_df['Primary Institution (Choose ONE)'].drop_duplicates().tolist()
affiliations = '("' + '"[ad] OR "'.join(affiliations_list) + '")'

concept = '("tubercul*"[tw] OR "Antitubercul*"[tw] OR "Anti-Tubercul*"[tw] OR "osteotubercul*"[tw] OR "nephrotubercul*"[tw] OR "anthracosilicotubercul*"[tw] OR "coniotubercul*"[tw] OR "Tuberculin"[tw] OR "tb"[tw] OR "xdr-tb"[tw] OR "xdrtb"[tw] OR "mdr-tb"[tw] OR "mdrtb"[tw] OR "phthisis"[tw] OR "pneumonophthisis"[tw] OR "pneumophthisiology"[tw] OR "silicotubercul*"[tw] OR "bazin disease"[tw] OR "erythema induratum"[tw] OR "white swelling"[tw] OR "king`s evil"[tw] OR "scrofula"[tw] OR "pott disease"[tw] OR "koch`s disease"[tw] OR "Interferon-gamma Release Test"[tw] OR "Tuberculosis"[Mesh] OR "Mycobacterium tuberculosis"[Mesh] OR "Antitubercular Agents"[Mesh] OR "Tuberculin Test"[Mesh] OR "Interferon-gamma Release Tests"[Mesh] OR "Tuberculosis Vaccines"[Mesh])'
concept_list = concept.\
    replace('[Mesh]', '[mh]').\
    split(' OR ')
concepts = ' OR '.join(concept_list)

#######################
# Format author names #
#######################

print('Formatting author names')

# Create new columns with e.g. "Ross J"[au] and "Ross JM"[au]
seatrac_df['LastFirst'] = '"' + seatrac_df['Last Name'] + ' ' + seatrac_df['First Name'].str[0] + '"' + '[au]'
seatrac_df['LastFirstMiddle'] = '"' + seatrac_df['Last Name'] + ' ' + seatrac_df['First Name'].str[0] + ' ' + seatrac_df['Middle Initial'].str[0] + '"' + '[au]'

# Combine new columns and format for pubmed
authors_list = seatrac_df['LastFirst'].dropna().to_list() + seatrac_df['LastFirstMiddle'].dropna().to_list()

#############
# Do search #
#############

print('Doing PubMed search')

def do_search(author_dat):
    '''
    Search two institutions at a time to keep URL from being too long.
    '''
    big_id_set = set()
    for i in range(0, len(affiliations_list), 2):
        affil_list = affiliations_list[i:i+2]
        affil = '("' + '"[ad] OR "'.join(affil_list) + '")'
        date_range = start_date + ':' + end_date + '[pdat]'
        url = prefix + date_range + '+AND+' + affil + '+AND+' + author_dat + '+AND+' + concepts
        page = requests.get(url).text
        result = BeautifulSoup(page, 'xml')
        IDlist = set([i.text for i in result.find_all('Id')])
        big_id_set = big_id_set.union(IDlist)
    return big_id_set

# Get number to cut author list in half (to search in sections to keep URL short)
half_authors_cutoff = int(len(authors_list)/2)

# First half
authors1 = '(' + ' OR '.join(authors_list[:half_authors_cutoff]) + ')'
id_set1 = do_search(authors1)

# Second half
authors2 = '(' + ' OR '.join(authors_list[half_authors_cutoff:]) + ')'
id_set2 = do_search(authors2)

out = ';'.join(id_set1.union(id_set2))

print('Done!\n')
print(f'PubMed IDs of papers from SEATRAC authors published between {start_date} and {end_date}:\n')
print(out)

Loading packages and data
Formatting affiliations and concepts
Formatting author names
Doing PubMed search
Done!

PubMed IDs of papers from SEATRAC authors published between 2024/04/01 and 2024/06/30:

38668542;38617280;38716731;38966641;39007063;38568690;38904070;38528148;38890283;38658786;39022779;38529729;38723342;39022782;38625902
